In [1]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt

all_data = pd.read_csv('sales所在周次节假日_剔除换货.csv')
all_data.rename(columns={'店铺名称':'store','地理位置指数':'position','年份':'years','周次':'weeks of year','是否包含法定节假日':'Isholiday','季节':'season','大类':'item','销量(剔除换货）':'sales','均价（剔除换货）':'prices','促销（剔除换货）':'discount'},inplace=True)
all_data['weeks'] = all_data['weeks of year']

In [2]:
a = all_data
week_5 = a.loc[lambda a : a['years'] == 2015]
week_6 = a.loc[lambda a : a['years'] == 2016]
week_6['weeks'] += 53
a = pd.concat([week_5,week_6])

/home/lin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [3]:
from sklearn import preprocessing

#map店铺名称
name = a['store']
le = preprocessing.LabelEncoder()
le.fit(name)
list(name)
le.transform(name)
tmp = le.transform(name)
a['store'] = tmp

#map合并大类
category = a['item']
le.fit(category)
list(category)
le.transform(category)
tmp = le.transform(category)
a['item'] = tmp

#map季节
season = a['season']
le.fit(season)
list(season)
le.transform(season)
tmp = le.transform(season)
a['season'] = tmp


In [4]:
#df = a
#masked_series = (df['years']==2016) & (df['weeks'].isin([53,54,55]))
#masked_series2 = (df['years']==2016) & (~(df['weeks'].isin([53,54,55])))
#df.loc[(masked_series), 'train_or_test'] = 'test'
#df.loc[(masked_series2), 'train_or_test'] = 'no_train'
#print('Train shape: {}'.format(df.loc[df['years']==2015,:].shape))
#print('Test shape: {}'.format(df.loc[df.train_or_test=='test',:].shape))


In [5]:
#df_n = df.drop(['train_or_test'],axis=1)
df_n = a

In [6]:
#df_n['sales'] = np.log1p(df.sales.values)
df_n['prices'] = np.log1p(df_n.sales.values)
df_n.sample(2)

,store,position,years,weeks of year,Isholiday,season,item,sales,prices,discount,weeks
14647,24,7,2016,4,0,0,2,31,3.465736,0.329595,57
8135,13,7,2015,32,0,2,6,10,2.397895,0.384358,32


In [7]:
df_n.describe()

,store,position,years,weeks of year,Isholiday,season,item,sales,prices,discount,weeks
count,18163.000000,18163.000000,18163.000000,18163.000000,18163.000000,18163.000000,18163.00000,18163.000000,18163.000000,18163.000000,18163.000000
mean,14.547156,5.501569,2015.552992,27.652701,0.185432,1.597864,5.41992,7.880031,1.697586,0.364135,56.961295
std,8.695615,2.597241,0.497198,15.181708,0.388658,1.048877,2.71015,16.436671,0.880375,0.089089,30.663260
min,0.000000,1.000000,2015.000000,1.000000,0.000000,0.000000,0.00000,1.000000,0.693147,0.032120,1.000000
25%,7.000000,7.000000,2015.000000,14.000000,0.000000,1.000000,3.00000,2.000000,1.098612,0.305955,31.000000
50%,14.000000,7.000000,2016.000000,28.000000,0.000000,2.000000,6.00000,4.000000,1.609438,0.368088,59.000000
75%,22.000000,7.000000,2016.000000,40.000000,0.000000,2.000000,7.00000,9.000000,2.302585,0.420665,84.000000
max,29.000000,7.000000,2016.000000,53.000000,1.000000,3.000000,11.00000,912.000000,6.816736,0.902023,106.000000


In [8]:
''''
def one_hot_encoder(df_n, ohe_cols=['store','item','weeks of years','isholiday','season','item']):

#    One-Hot Encoder function
    print('Creating OHE features..\nOld df shape:{}'.format(df.shape))
    df_n = pd.get_dummies(df_n, columns=ohe_cols)
    print('New df shape:{}'.format(df.shape))
    return df_n
'''

"'\ndef one_hot_encoder(df_n, ohe_cols=['store','item','weeks of years','isholiday','season','item']):\n\n#    One-Hot Encoder function\n    print('Creating OHE features..\nOld df shape:{}'.format(df.shape))\n    df_n = pd.get_dummies(df_n, columns=ohe_cols)\n    print('New df shape:{}'.format(df.shape))\n    return df_n\n"

In [9]:
#df_n = df_n.drop(['prices','discount'],axis=1)

In [10]:
train = df_n[lambda df_n: df_n['years']==2015]
t1 = df_n[lambda df_n: df_n['weeks']==57]
t2 = df_n[lambda df_n: df_n['weeks']==58]
t3 = df_n[lambda df_n: df_n['weeks']==59]
#test = pd.concat([t1,t2,t3])

In [11]:
all_test = df_n[lambda df_n: df_n['years'] == 2016]
test = all_test[lambda all_test: all_test['weeks'] <82]
val_test = all_test[lambda all_test: all_test['weeks'] >82]
#test = tmp[lambda tmp: tmp['weeks']<65]

In [12]:
#train = one_hot_encoder(train, ohe_cols=['store','item','weeks of year','Isholiday','season','item']) 

In [13]:
y_train = train['sales']
y_test = test['sales']
X_train = train.drop('sales', axis=1)
X_test = test.drop('sales', axis=1)
y_val_test = val_test['sales']
X_val_test = val_test.drop('sales',axis=1)

In [14]:
import lightgbm as lgb
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

ModuleNotFoundError: No module named 'lightgbm'

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

print('Saving model...')
# save model to file
gbm.save_model('model.txt')

print('Starting predicting...')
# predict
y_pred = gbm.predict(X_val_test, num_iteration=gbm.best_iteration)
# eval
print('The rmse of prediction is:', mean_squared_error(y_val_test, y_pred) ** 0.5)
print('The R^2 of prediction is:', r2_score(y_val_test, y_pred))

In [ ]:
y_pred.shape

In [ ]:
from sklearn import datasets, linear_model
# Load the diabetes dataset
diabetes = datasets.load_diabetes()


# Use only one feature
diabetes_X = diabetes.data[:, np.newaxis, 2]

# Split the data into training/testing sets
#diabetes_X_train = diabetes_X[:-20]
#diabetes_X_test = diabetes_X[-20:]
diabetes_X_train = df_n[lambda df_n: df_n['years'] == 2015][['weeks','prices']]
diabetes_X_test = df_n[lambda df_n: df_n['years'] == 2016][['weeks','prices']]

# Split the targets into training/testing sets
#diabetes_y_train = diabetes.target[:-20]
#diabetes_y_test = diabetes.target[-20:]
diabetes_y_train = df_n[lambda df_n: df_n['years'] == 2015]['sales']
diabetes_y_test = df_n[lambda df_n: df_n['years'] == 2016]['sales']
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(diabetes_X_train, diabetes_y_train)

# Make predictions using the testing set
diabetes_y_pred = regr.predict(diabetes_X_test)

# The coefficients
print('Coefficients:', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(diabetes_y_test, diabetes_y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(diabetes_y_test, diabetes_y_pred))


In [ ]:
from sklearn import datasets, linear_model
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn import datasets

diabetes = datasets.load_diabetes()
diabetes_X = diabetes.data[:, np.newaxis, 8]

regL = linear_model.Lasso
regR = linear_model.Ridge

regL = linear_model.Lasso(alpha=0.1)
df_nR = df_n[lambda df_n: df_n['years'] == 2015].drop(['sales'],axis=1)
#diabetes_X_train = df_n[lambda df_n: df_n['years'] == 2015]
df_nR = df_n[lambda df_n: df_n['years'] == 2016].drop(['sales'],axis=1)
#diabetes_X_test = df_n[lambda df_n: df_n['years'] == 2016]

diabetes_y_train = df_n[lambda df_n: df_n['years'] == 2015]['sales']
diabetes_y_test = df_n[lambda df_n: df_n['years'] == 2016]['sales']

regL.fit(diabetes_X_train, diabetes_y_train,[0,1])
#regR.fit(diabetes_X_train, diabetes_y_train,[0,1])

diabetes_y_pred = regL.predict(diabetes_X_test)
#diabetes_y_pred = regR.predict(diabetes_X_test)


# The coefficients
print('Coefficients:', regL.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(diabetes_y_test, diabetes_y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(diabetes_y_test, diabetes_y_pred))